In [2]:
import json
import csv
import pandas as pd
from py2neo import Graph
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import random as rd

In [16]:
# Establish a connection to the Neo4j database
graph = Graph("bolt://localhost:7687", auth=("user_name", "user_password"))
# Generate random integers for graph names
rd_int1 = rd.randint(1,500)
rd_int2 = rd.randint(500,9999)
# Create graph names
graph1 = 'g' + str(rd_int1)
graph2 = 'g' + str(rd_int2)

In [4]:
# Read data from a CSV file
df = pd.read_csv('title_list.csv')
print(df)

                                   현수교 교량공사 안전보건작업 지침
0                     소규모 철근콘크리트 교량공사 거푸집 동바리 안전작업 지침
1                         교량공사의 이동식 비계공법(MSS) 안전작업 지침
2                                      낙하물 방호선반 설치 지침
3                            흙막이공사(엄지말뚝 공법) 안전보건작업 지침
4                철골공사 무지보 거푸집동바리(데크플레이트 공법) 안전보건작업 지침
5                    흙막이공사(Soil Nailing 공법) 안전보건작업 지침
6                    흙막이공사(Earth Anchor 공법) 안전보건작업 지침
7                          관로매설공사(유압식 추진공법) 안전보건작업 지침
8                                   취약시기 건설현장 안전작업 지침
9                  해상 RCD 현장타설 말뚝공사(현수교, 사장교) 안전작업 지침
10                               PCT거더 교량공사 안전보건작업 지침
11                               트러스거더 교량공사 안전보건작업 지침
12                               덤프트럭 및 화물자동차 안전작업 지침
13                                       강관비계 안전작업 지침
14                                   조적공사 안전보건작업 기술지침
15                        중소규모 건설업체 본사의 안전보건관리에 관한 지침
16                                   수직형 추락방망 설치 기술지침
17                         건

In [17]:
# Prompt the user for input
title = str(input("input Title : "))
index = str(input("input Index : "))
print(title)
print(index)

파이프 서포트 동바리 안전작업 지침
거푸집


In [18]:
# Construct and execute a Cypher query to project a subgraph from graph1
query1 = f'''CALL gds.graph.project.cypher('{graph1}','MATCH (n1:Index{{name:"{index}"}})--(n2:Content)--(n3:Category)--(n4:Content{{Title:"{title}"}}) RETURN id(n4) AS id',
  'MATCH (n4)-[r:Relate]-(n5:Content{{Title:"{title}"}}) RETURN id(n4) AS source, id(n5) AS target, type(r) AS type, count(r) AS numberOfReads',
  {{ validateRelationships: false }}
)
YIELD graphName AS graph, nodeCount AS nodes, relationshipCount AS rels;'''
result = graph.run(query1).data()
print(result)

[{'graph': 'g320', 'nodes': 38, 'rels': 1194}]


In [19]:
# Construct and execute a Cypher query to project a subgraph from graph2
query2 = f'''MATCH (source)
WHERE source:Content
OPTIONAL MATCH (source:Content{{Title:"{title}"}})-[r:Include]->(target)
WHERE target:Index WITH gds.alpha.graph.project(
  '{graph2}',
  source,
  target,
  {{
    sourceNodeLabels: labels(source),
    targetNodeLabels: labels(target)
  }},
  {{
    relationshipType: type(r)
  }}
) AS g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels'''
result = graph.run(query2).data()
print(result)

[{'graph': 'g8941', 'nodes': 7098, 'rels': 1419}]


In [20]:
# Execute PageRank algorithm on graph1
query3 = f'''CALL gds.pageRank.write('{graph1}',
{{maxIterations: 100, dampingFactor: 0.85, writeProperty: '{graph1}'}})
YIELD nodePropertiesWritten, ranIterations'''
result = graph.run(query3).data()
print(result)

[{'nodePropertiesWritten': 38, 'ranIterations': 87}]


In [21]:
# Execute Louvain algorithm on graph2
query4 = f'''call gds.louvain.write('{graph2}',{{writeProperty: '{graph2}'}})'''
result = graph.run(query4).data()
print(result)

[{'writeMillis': 51, 'nodePropertiesWritten': 7098, 'modularity': -0.04215963658414127, 'modularities': [-0.2028426255772733, -0.1817009765286475, -0.15850624838284016, -0.13947876423907696, -0.12097820726563539, -0.10547433129664316, -0.08726182122120962, -0.07175794525221736, -0.056958790918179326, -0.04215963658414127], 'ranLevels': 10, 'communityCount': 6863, 'communityDistribution': {'p99': 1, 'min': 1, 'max': 25, 'mean': 1.0342415853125455, 'p90': 1, 'p50': 1, 'p999': 11, 'p95': 1, 'p75': 1}, 'postProcessingMillis': 2, 'preProcessingMillis': 0, 'computeMillis': 1679, 'configuration': {'maxIterations': 10, 'writeConcurrency': 4, 'seedProperty': None, 'consecutiveIds': False, 'maxLevels': 10, 'concurrency': 4, 'jobId': '0e9a6650-936b-410b-b5ef-83758170860f', 'writeProperty': 'g8941', 'logProgress': True, 'includeIntermediateCommunities': False, 'nodeLabels': ['*'], 'sudo': False, 'relationshipTypes': ['*'], 'tolerance': 0.0001}}]


In [22]:
# Retrieve nodes from the graph based on the query
query = f'''MATCH (n1:Index{{name:"{index}"}})--(n2:Content)--(n3:Category)--(n4:Content{{Title:"{title}"}})
return n4'''
result = graph.run(query).data()


unique_nodes = list({node['n4']['Content']: node for node in result}.values())

print(unique_nodes)

# Get unique nodes based on their content property
louvain_list = []

for node in result:
    if node['n4'].get(f'{graph2}', 0) != 0:
        louvain_list.append(node['n4'][f'{graph2}'])

louvain_list = list(set(louvain_list))
print(louvain_list)


[{'n4': Node('Content', Category='일반안전사항', Content='재료, 기구 또는 공구 등을 올리거나 내리는 경우에는 건설장비 사용을 원착으로 하며, 부치나 길이가 작은 재료, 기구 또는 공구 등은 양중함이나 달줄ㆍ달포대 등을 사용하도록 하여야 한다.', Index=['건설장비', '건설장비 사용을', '경우에는', '경우에는 건설장비', '공구', '공구 등은', '공구 등을', '기구', '기구 또는', '길이가', '길이가 작은', '내리는', '내리는 경우에는', '달줄ㆍ달포대', '달줄ㆍ달포대 등을', '등은', '등은 양중함이나', '등을', '등을 사용하도록', '등을 올리거나', '또는', '또는 공구', '부치나', '부치나 길이가', '사용을', '사용을 원착으로', '사용하도록', '사용하도록 한다', '양중함이나', '양중함이나 달줄ㆍ달포대', '올리거나', '올리거나 내리는', '원착으로', '원착으로 하며', '작은', '작은 재료', '재료', '재료 기구', '하며', '하며 부치나', '한다'], Title='파이프 서포트 동바리 안전작업 지침', g320=1.0969486287836412, g5576=8102, g8668=8102, g8941=4950)}, {'n4': Node('Content', Category='일반안전사항', Content='파이프 서포트 동바리 재료는 고용노동부고시 제2020-33호(방호장치 안전인증 고시)에 적합한 것을 사용하여야 한다', Index=['33호', '33호 방호장치', '것을', '것을 사용하여야', '고시', '고시 적합한', '고용노동부고시', '고용노동부고시 제2020', '동바리', '동바리 재료는', '방호장치', '방호장치 안전인증', '사용하여야', '사용하여야 한다', '서포트', '서포트 동바리', '안전인증', '안전인증 고시', '재료는', '재료는 고용노동부고시', '적합한', '적합한 것을', '제2020', '제2020 33호', '파

In [24]:
# Print nodes grouped by Louvain values
for i in louvain_list:
    group_0 = [node for node in result if node['n4'][f'{graph2}'] == i]
    group_0_sorted = sorted(group_0, key=lambda node: node['n4'].get(f'{graph1}', 0), reverse=True)
    unique_nodes = set()
    for node in group_0_sorted:
        if node['n4'].get(f'{graph2}') is not None and node['n4'][f'{graph2}'] >= 0.9:
            unique_nodes.add(node['n4']['Content'])
    iteration = 0
    for node in unique_nodes:
        iteration += 1
        if iteration >= 2:
            print(node)
    print('--------------------------------')


--------------------------------
--------------------------------
--------------------------------
--------------------------------
--------------------------------
해체 시기⋅범위 및 절차 등의 해체작업 안전계획은 해체작업 근로자에게 교육 하여야 한다
거푸집 동바리 해체작업은 작업전 해체 시기 순서 등 해체작업 안전계획을 수 립하여야 한다.
--------------------------------
--------------------------------
--------------------------------
--------------------------------
--------------------------------
--------------------------------
콘크리트 타설작업은 작업전 청소 실시하고 조립도 준수 여부를 확인하여야 한 다.
--------------------------------
파이프 서포트 동바리는 구조를 검토한 후 조립도를 작성하고, 조립도에 따라 조 립하여야 한다
경사면 하부에 조립하는 거푸집 동바리는 다음 사항을 준수하여야 한다. 가. 거푸집의 형상에 따른 부득이한 경우를 제외하고는 깔판ㆍ깔목 등을 2단 이상 끼우지 않도록 할 것 나. 깔판ㆍ깔목 등을 이어서 사용하는 경우에는 깔판ㆍ깔목은 단단히 연결할 것 다. 경사면에 설치하는 동바리는 연직도를 유지하도록 깔판ㆍ깔목 등으로 고정할 것 라. 연직하게 설치되는 동바리는 경사면방향 분력으로 인하여 미끄러짐 및 전도가 발생할 수 있으므로 모든 동바리에 가새를 설치하는 등 안전조치할 것.
개구부 상부에 동바리를 설치하는 경우에는 상부하중을 견딜 수 있는 견고한 받침대를 설치하여야 한다.
파이프 서포트 동바리는 다음 사항을 준수하여야 한다. 가. 파이프 서포트를 3개 이상 이어서 사용하지 않도록 할 것 나. 파이프 서포트를 이어서 사용하

In [25]:
# Remove graph properties from the nodes
query1 = f'''MATCH (n:content) REMOVE n.{graph1}'''
query2 = f'''MATCH (n:content) REMOVE n.{graph2}'''

graph.run(query1)
graph.run(query2)

(No data)

In [26]:
# Drop the projected graphs
query1 = f"CALL gds.graph.drop('{graph1}')"
query2 = f"CALL gds.graph.drop('{graph2}')"
result1 = graph.run(query1)
result2 = graph.run(query2)
print(result1)
print(result2)

 graphName | database | memoryUsage | sizeInBytes | nodeCount | relationshipCount | configuration                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |            density | creationTime                                    | modificationTime                                | schema                                                                                                           